In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
from tqdm import tqdm
import re


from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import log_loss,roc_auc_score
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer
import pickle
from sklearn.linear_model import LogisticRegression
le = LabelEncoder()

In [2]:
train_doc = pd.read_pickle('../../data/fe_doc/train_doc.pkl')
test_doc = pd.read_pickle('../../data/fe_doc/test_doc.pkl')

# TFIDF

In [3]:

holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_index = pickle.load(open(FILE.train_index.value,'rb'))
train_cv = train_doc.loc[train_index].copy()
holdout = train_doc.loc[holdout_index].copy()

In [5]:
tft = TfidfVectorizer(token_pattern=r'(?u)\b[\w.-]+\b',
                max_features=500000)
tfidf = tft.fit(pd.concat([train_doc['text'],test_doc['text']]))

holdout_matrix = tfidf.transform(holdout['text'])
test_matrix = tfidf.transform(test_doc['text'])

In [12]:
n_fold = 5
sf = StratifiedKFold(n_splits=5,shuffle= True,random_state=20)
train_index_list = []
val_index_list = []
for t,v in sf.split(train_cv,train_cv.click):
    train_index_list.append(t)
    val_index_list.append(v)

In [13]:
train_cv.shape

(961626, 2)

In [14]:
def cross_validation(train_set,train_index_list,val_index_list,tfidf,holdout_x,holdout_y,test_x):
    n_fold = len(train_index_list)
    roc_list = []
    log_list = []
    hold_out_score_list = []
    hold_out_preds_list = []
    test_list = []
    for fold in range(n_fold):
        train_x = tfidf.transform(train_set.iloc[train_index_list[fold]]['text'])
        train_y = train_set.iloc[train_index_list[fold]]['click'].values
        print('done train tfidf')
        val_x = tfidf.transform(train_set.iloc[val_index_list[fold]]['text'])
        val_y = train_set.iloc[val_index_list[fold]]['click'].values
        print('done val tfidf')
        

    #     l_svc = LinearSVC()
    #     lsvc = CalibratedClassifierCV(l_svc) 
    #     lsvc.fit(train_x, train_y)
    #     preds = lsvc.predict_proba(val_x)[:,1]

        lg = LogisticRegression()
        lg.fit(train_x, train_y)
        preds = lg.predict_proba(val_x)[:,1]
        preds_holdout = lg.predict_proba(holdout_x)[:,1]
        preds_test = lg.predict_proba(test_x)[:,1]

        log_score = log_loss(val_y,preds)
        roc_score = roc_auc_score(val_y,preds)
        log_score_ho = log_loss(holdout_y,preds_holdout)
        roc_score_ho = roc_auc_score(holdout_y,preds_holdout)
        print('fold {} ends. Logloss is: {}. Roc score is: {}'.format(fold,log_score,roc_score))
        print('fold {} ends. Logloss holdout is: {}. Roc score holdout is: {}'.format(fold,log_score_ho,roc_score_ho))

        log_list.append(log_score)
        roc_list.append(roc_score)
        hold_out_preds_list.append(preds_holdout)
        hold_out_score_list.append(log_score_ho)
        test_list.append(preds_test)
    ho = np.mean(hold_out_preds_list,axis=0)
    ho_overall = log_loss(holdout_y,ho)
    test_return = np.mean(test_list,axis=0)
    print('{} fold logloss is: {}'.format(n_fold,np.mean(log_list)))
    print('{} fold roc is: {}'.format(n_fold,np.mean(roc_list)))
    print('{} fold mean hold out is: {}'.format(n_fold,np.mean(hold_out_score_list)))
    print('{} fold overall hold out is: {}'.format(n_fold,ho_overall))
    return ho,test_return

In [15]:
ho_log_preds,test_log_preds = cross_validation(train_cv,train_index_list,val_index_list,tfidf,holdout_matrix,holdout.click.values,test_matrix)

done train tfidf
done val tfidf
fold 0 ends. Logloss is: 0.419051953152452. Roc score is: 0.7627029220582975
fold 0 ends. Logloss holdout is: 0.4297131504702662. Roc score holdout is: 0.762086938254896
done train tfidf
done val tfidf
fold 1 ends. Logloss is: 0.41781310251249104. Roc score is: 0.7651620714044061
fold 1 ends. Logloss holdout is: 0.4292372642829562. Roc score holdout is: 0.7628263468081055
done train tfidf
done val tfidf


KeyboardInterrupt: 

In [14]:
ho_log_preds,test_log_preds = cross_validation(train_cv,train_index_list,val_index_list,tfidf,holdout_matrix,holdout.click.values,test_matrix)

done train tfidf
done val tfidf
fold 0 ends. Logloss is: 0.41911197470877354. Roc score is: 0.762721805982848
fold 0 ends. Logloss holdout is: 0.4299107188509282. Roc score holdout is: 0.7617711656041624
done train tfidf
done val tfidf
fold 1 ends. Logloss is: 0.4178221464163864. Roc score is: 0.7652451855650636
fold 1 ends. Logloss holdout is: 0.42943473711744334. Roc score holdout is: 0.7625517174065715
done train tfidf
done val tfidf
fold 2 ends. Logloss is: 0.4183649923295717. Roc score is: 0.7639747046214409
fold 2 ends. Logloss holdout is: 0.4296427213866519. Roc score holdout is: 0.7618465792200266
done train tfidf
done val tfidf
fold 3 ends. Logloss is: 0.41944988731000094. Roc score is: 0.7630068469643847
fold 3 ends. Logloss holdout is: 0.4293827018127397. Roc score holdout is: 0.7626829153547374
done train tfidf
done val tfidf
fold 4 ends. Logloss is: 0.41823477969576794. Roc score is: 0.7649211593807996
fold 4 ends. Logloss holdout is: 0.42959019260064457. Roc score holdout

In [15]:
def cross_validation_svc(train_set,train_index_list,val_index_list,tfidf,holdout_x,holdout_y,test_x):
    n_fold = len(train_index_list)
    roc_list = []
    log_list = []
    hold_out_score_list = []
    hold_out_preds_list = []
    test_list = []
    for fold in range(n_fold):
        train_x = tfidf.transform(train_set.iloc[train_index_list[fold]]['text'])
        train_y = train_set.iloc[train_index_list[fold]]['click'].values
        print('done train tfidf')
        val_x = tfidf.transform(train_set.iloc[val_index_list[fold]]['text'])
        val_y = train_set.iloc[val_index_list[fold]]['click'].values
        print('done val tfidf')
        

        l_svc = LinearSVC()
        lsvc = CalibratedClassifierCV(l_svc) 
        lsvc.fit(train_x, train_y)
        preds = lsvc.predict_proba(val_x)[:,1]
        preds_holdout = lsvc.predict_proba(holdout_x)[:,1]
        preds_test = lsvc.predict_proba(test_x)[:,1]

#         lg = LogisticRegression()
#         lg.fit(train_x, train_y)
#         preds = lg.predict_proba(val_x)[:,1]
#         preds_holdout = lg.predict_proba(holdout_x)[:,1]
#         preds_test = lg.predict_proba(test_x)[:,1]

        log_score = log_loss(val_y,preds)
        roc_score = roc_auc_score(val_y,preds)
        log_score_ho = log_loss(holdout_y,preds_holdout)
        roc_score_ho = roc_auc_score(holdout_y,preds_holdout)
        print('fold {} ends. Logloss is: {}. Roc score is: {}'.format(fold,log_score,roc_score))
        print('fold {} ends. Logloss holdout is: {}. Roc score holdout is: {}'.format(fold,log_score_ho,roc_score_ho))

        log_list.append(log_score)
        roc_list.append(roc_score)
        hold_out_preds_list.append(preds_holdout)
        hold_out_score_list.append(log_score_ho)
        test_list.append(preds_test)
    ho = np.mean(hold_out_preds_list,axis=0)
    ho_overall = log_loss(holdout_y,ho)
    test_return = np.mean(test_list,axis=0)
    print('{} fold logloss is: {}'.format(n_fold,np.mean(log_list)))
    print('{} fold roc is: {}'.format(n_fold,np.mean(roc_list)))
    print('{} fold mean hold out is: {}'.format(n_fold,np.mean(hold_out_score_list)))
    print('{} fold overall hold out is: {}'.format(n_fold,ho_overall))
    return ho,test_return

In [16]:
ho_svc_preds,test_svc_preds = cross_validation_svc(train_cv,train_index_list,val_index_list,tfidf,holdout_matrix,holdout.click.values,test_matrix)

done train tfidf
done val tfidf
fold 0 ends. Logloss is: 0.4265399266021514. Roc score is: 0.7595579482509007
fold 0 ends. Logloss holdout is: 0.43807563342576433. Roc score holdout is: 0.7583700028764536
done train tfidf
done val tfidf
fold 1 ends. Logloss is: 0.42559086250587. Roc score is: 0.7620663902085789
fold 1 ends. Logloss holdout is: 0.43747327868739544. Roc score holdout is: 0.7595119245979104
done train tfidf
done val tfidf
fold 2 ends. Logloss is: 0.42585441982601585. Roc score is: 0.7606007759513209
fold 2 ends. Logloss holdout is: 0.43780144744928995. Roc score holdout is: 0.7587191769087037
done train tfidf
done val tfidf
fold 3 ends. Logloss is: 0.4266954173483883. Roc score is: 0.7595487981777704
fold 3 ends. Logloss holdout is: 0.437141558764647. Roc score holdout is: 0.7598632278318653
done train tfidf
done val tfidf
fold 4 ends. Logloss is: 0.4252691830828662. Roc score is: 0.7616945787534134
fold 4 ends. Logloss holdout is: 0.437578919400539. Roc score holdout is:

In [25]:
df_ho_log = pd.DataFrame({'preds':ho_log_preds,'instance_id':train.loc[holdout_index,'instance_id'].values})
df_test_log = pd.DataFrame({'predicted_score':test_log_preds,'instance_id':test.instance_id.values})

df_ho_svc = pd.DataFrame({'preds':ho_svc_preds,'instance_id':train.loc[holdout_index,'instance_id'].values})
df_test_svc = pd.DataFrame({'predicted_score':test_svc_preds,'instance_id':test.instance_id.values})


df_ho_log.to_csv('ho_logistic.csv',index=False)
df_test_log.to_csv('test_logistic.csv',index=False)
df_ho_svc.to_csv('ho_svc.csv',index=False)
df_test_svc.to_csv('test_svc.csv',index=False)

# count vectorizer

In [26]:
tft = CountVectorizer(token_pattern=r'(?u)\b[\w.-]+\b',
                max_features=500000)
tfidf = tft.fit(all_doc)

holdout_matrix = tfidf.transform(holdout['text'])
test_matrix = tfidf.transform(test_doc['text'])

In [27]:
ho_log_preds_cou,test_log_preds_cou = cross_validation(train_cv,train_index_list,val_index_list,tfidf,holdout_matrix,holdout.click.values,test_matrix)

done train tfidf
done val tfidf
fold 0 ends. Logloss is: 0.4188224224745025. Roc score is: 0.7629093614357909
fold 0 ends. Logloss holdout is: 0.4295705655082061. Roc score holdout is: 0.7621863260800635
done train tfidf
done val tfidf
fold 2 ends. Logloss is: 0.41806067303521943. Roc score is: 0.7643210836349276
fold 2 ends. Logloss holdout is: 0.4290855798087492. Roc score holdout is: 0.7626188324023631
done train tfidf
done val tfidf
fold 3 ends. Logloss is: 0.41918833284970286. Roc score is: 0.7638357345378217
fold 3 ends. Logloss holdout is: 0.4287456275231553. Roc score holdout is: 0.7633409139183763
done train tfidf
done val tfidf
fold 4 ends. Logloss is: 0.41805114009571753. Roc score is: 0.7653429564285393
fold 4 ends. Logloss holdout is: 0.4289274618035679. Roc score holdout is: 0.7628976223994547
5 fold logloss is: 0.41827702933901617
5 fold roc is: 0.7644278156536235
5 fold mean hold out is: 0.4290227418639551
5 fold overall hold out is: 0.42828574043402995


In [30]:
df_ho_log = pd.DataFrame({'preds':ho_log_preds_cou,'instance_id':train.loc[holdout_index,'instance_id'].values})
df_test_log = pd.DataFrame({'predicted_score':test_log_preds_cou,'instance_id':test.instance_id.values})



df_ho_log.to_csv('../../data/Blend/ho_logistic.csv',index=False)
df_test_log.to_csv('../../data/Blend/test_logistic.csv',index=False)


In [28]:
ho_svc_preds_cou,test_svc_preds_cou = cross_validation_svc(train_cv,train_index_list,val_index_list,tfidf,holdout_matrix,holdout.click.values,test_matrix)

done train tfidf
done val tfidf
fold 0 ends. Logloss is: 0.42613729014720214. Roc score is: 0.7594076639861963
fold 0 ends. Logloss holdout is: 0.43724957130641245. Roc score holdout is: 0.7590066078533506
done train tfidf
done val tfidf


KeyboardInterrupt: 